In [5]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
from scipy import stats
# import tensorflow as tf

In [6]:
# Import our mental health dataset and create a dataframe
mental_df = pd.read_csv('osmi 2016.csv')
mental_df

,Are you self-employed?,How many employees does your company or organization have?,Is your employer primarily a tech company/organization?,Is your primary role within your company related to tech/IT?,Does your employer provide mental health benefits as part of healthcare coverage?,Do you know the options for mental health care available under your employer-provided coverage?,"Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",Does your employer offer resources to learn more about mental health concerns and options for seeking help?,Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?,"If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:",...,"If you have a mental health issue, do you feel that it interferes with your work when being treated effectively?","If you have a mental health issue, do you feel that it interferes with your work when NOT being treated effectively?",What is your age?,What is your gender?,What country do you live in?,What US state or territory do you live in?,What country do you work in?,What US state or territory do you work in?,Which of the following best describes your work position?,Do you work remotely?
0,0,26-100,1.0,NaN,Not eligible for coverage / N/A,NaN,No,No,I don't know,Very easy,...,Not applicable to me,Not applicable to me,39,Male,United Kingdom,NaN,United Kingdom,NaN,Back-end Developer,Sometimes
1,0,25-Jun,1.0,NaN,No,Yes,Yes,Yes,Yes,Somewhat easy,...,Rarely,Sometimes,29,male,United States of America,Illinois,United States of America,Illinois,Back-end Developer|Front-end Developer,Never
2,0,25-Jun,1.0,NaN,No,NaN,No,No,I don't know,Neither easy nor difficult,...,Not applicable to me,Not applicable to me,38,Male,United Kingdom,NaN,United Kingdom,NaN,Back-end Developer,Always
3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Sometimes,Sometimes,43,male,United Kingdom,NaN,United Kingdom,NaN,Supervisor/Team Lead,Sometimes
4,0,25-Jun,0.0,1.0,Yes,Yes,No,No,No,Neither easy nor difficult,...,Sometimes,Sometimes,43,Female,United States of America,Illinois,United States of America,Illinois,Executive Leadership|Supervisor/Team Lead|Dev ...,Sometimes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1428,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Not applicable to me,Not applicable to me,34,Female,United States of America,New York,United States of America,New York,Other,Sometimes
1429,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Sometimes,Often,56,MALE,United States of America,California,Afghanistan,NaN,Support,Sometimes
1430,0,100-500,1.0,NaN,Yes,Yes,Yes,Yes,I don't know,Somewhat difficult,...,Rarely,Sometimes,52,Male,United States of America,Georgia,United States of America,Georgia,Back-end Developer,Sometimes
1431,0,100-500,0.0,1.0,I don't know,I am not sure,No,Yes,I don't know,Somewhat difficult,...,Sometimes,Often,30,Female,United States of America,Nebraska,United States of America,Nebraska,DevOps/SysAdmin,Sometimes


In [7]:
mental_df.shape

(1433, 63)

In [8]:
mental_df = mental_df.replace({"Yes": 1, "No": -1,'Maybe':0,"I don't know":0})


In [9]:
mental_df

,Are you self-employed?,How many employees does your company or organization have?,Is your employer primarily a tech company/organization?,Is your primary role within your company related to tech/IT?,Does your employer provide mental health benefits as part of healthcare coverage?,Do you know the options for mental health care available under your employer-provided coverage?,"Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",Does your employer offer resources to learn more about mental health concerns and options for seeking help?,Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?,"If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:",...,"If you have a mental health issue, do you feel that it interferes with your work when being treated effectively?","If you have a mental health issue, do you feel that it interferes with your work when NOT being treated effectively?",What is your age?,What is your gender?,What country do you live in?,What US state or territory do you live in?,What country do you work in?,What US state or territory do you work in?,Which of the following best describes your work position?,Do you work remotely?
0,0,26-100,1.0,NaN,Not eligible for coverage / N/A,NaN,-1.0,-1.0,0.0,Very easy,...,Not applicable to me,Not applicable to me,39,Male,United Kingdom,NaN,United Kingdom,NaN,Back-end Developer,Sometimes
1,0,25-Jun,1.0,NaN,-1,1,1.0,1.0,1.0,Somewhat easy,...,Rarely,Sometimes,29,male,United States of America,Illinois,United States of America,Illinois,Back-end Developer|Front-end Developer,Never
2,0,25-Jun,1.0,NaN,-1,NaN,-1.0,-1.0,0.0,Neither easy nor difficult,...,Not applicable to me,Not applicable to me,38,Male,United Kingdom,NaN,United Kingdom,NaN,Back-end Developer,Always
3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Sometimes,Sometimes,43,male,United Kingdom,NaN,United Kingdom,NaN,Supervisor/Team Lead,Sometimes
4,0,25-Jun,0.0,1.0,1,1,-1.0,-1.0,-1.0,Neither easy nor difficult,...,Sometimes,Sometimes,43,Female,United States of America,Illinois,United States of America,Illinois,Executive Leadership|Supervisor/Team Lead|Dev ...,Sometimes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1428,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Not applicable to me,Not applicable to me,34,Female,United States of America,New York,United States of America,New York,Other,Sometimes
1429,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Sometimes,Often,56,MALE,United States of America,California,Afghanistan,NaN,Support,Sometimes
1430,0,100-500,1.0,NaN,1,1,1.0,1.0,0.0,Somewhat difficult,...,Rarely,Sometimes,52,Male,United States of America,Georgia,United States of America,Georgia,Back-end Developer,Sometimes
1431,0,100-500,0.0,1.0,0,I am not sure,-1.0,1.0,0.0,Somewhat difficult,...,Sometimes,Often,30,Female,United States of America,Nebraska,United States of America,Nebraska,DevOps/SysAdmin,Sometimes


In [11]:
# Check for low response rate columns < 70% response rate
low_response_columns = []
for column in mental_df.columns:
    if ((mental_df[column].count() / 1433) < .8):
        low_response_columns.append(column)

low_response_columns

['How many employees does your company or organization have?',
 'Is your employer primarily a tech company/organization?',
 'Is your primary role within your company related to tech/IT?',
 'Does your employer provide mental health benefits as part of healthcare coverage?',
 'Do you know the options for mental health care available under your employer-provided coverage?',
 'Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?',
 'Does your employer offer resources to learn more about mental health concerns and options for seeking help?',
 'Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?',
 'If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:',
 'Do you think that discussing a mental health disorder with your employer would have negative consequences?',
 'Do 

In [12]:
# drop low response columns
mental_df.drop(columns=low_response_columns, inplace=True)
mental_df.shape

(1433, 32)

In [13]:
# Check for many distinct answers indicating long for response or irrelavant data
many_distinct_columns = []
for column in mental_df.columns:
    if (mental_df[column].nunique() > 10):
        many_distinct_columns.append(column)
    

many_distinct_columns.remove('What is your age?')
many_distinct_columns.remove('What country do you live in?')
many_distinct_columns.remove('What country do you work in?')
many_distinct_columns.remove('What is your gender?')



many_distinct_columns

['Which of the following best describes your work position?']

In [14]:
# drop many distinct answers columns
mental_df.drop(columns=many_distinct_columns, inplace=True)
mental_df.shape

(1433, 31)

In [15]:
# Fix gender data to be consistent
mental_df['What is your gender?'] = mental_df['What is your gender?'].replace('male', 'Male')
mental_df['What is your gender?'] = mental_df['What is your gender?'].replace('m', 'Male')
mental_df['What is your gender?'] = mental_df['What is your gender?'].replace('M', 'Male')
mental_df['What is your gender?'] = mental_df['What is your gender?'].replace('f', 'Male')
mental_df['What is your gender?'] = mental_df['What is your gender?'].replace('F', 'Female')
mental_df['What is your gender?'] = mental_df['What is your gender?'].replace('female', 'Female')

In [16]:
# Print out gender counts
gender_counts = mental_df['What is your gender?'].value_counts()
gender_counts

Male           1047
Female          286
Male             11
Female            9
non-binary        4
               ... 
genderqueer       1
cis male          1
Genderfluid       1
Enby              1
MALE              1
Name: What is your gender?, Length: 64, dtype: int64

In [17]:
# Determine which values to replace
replace_genders = list(gender_counts[gender_counts < 20].index)

# Replace in DataFrame
for gender in replace_genders:
    mental_df['What is your gender?'] = mental_df['What is your gender?'].replace(gender,"Other")

In [18]:
# Print out gender counts
gender_counts = mental_df['What is your gender?'].value_counts()
gender_counts

Male      1047
Female     286
Other       97
Name: What is your gender?, dtype: int64

In [19]:
# Print out country counts
country_counts = mental_df['What country do you live in?'].value_counts()
country_counts

United States of America    840
United Kingdom              180
Canada                       78
Germany                      58
Netherlands                  48
Australia                    35
Sweden                       19
France                       16
Ireland                      15
Switzerland                  10
Brazil                       10
New Zealand                   9
India                         9
Russia                        9
Bulgaria                      7
Finland                       7
Denmark                       7
Italy                         5
Belgium                       5
South Africa                  4
Romania                       4
Spain                         4
Poland                        4
Austria                       4
Norway                        3
Pakistan                      3
Chile                         3
Czech Republic                3
Japan                         2
Israel                        2
Other                         2
Bosnia a

In [20]:
# Determine which values to replace
replace_countries = list(country_counts[country_counts < 20].index)

# Replace in DataFrame
for country in replace_countries:
    mental_df['What country do you live in?'] = mental_df['What country do you live in?'].replace(country,"Other")

In [21]:
# Print out country counts
country_counts = mental_df['What country do you live in?'].value_counts()
country_counts

United States of America    840
Other                       194
United Kingdom              180
Canada                       78
Germany                      58
Netherlands                  48
Australia                    35
Name: What country do you live in?, dtype: int64

In [22]:
# Print out country counts
country_counts = mental_df['What country do you work in?'].value_counts()
country_counts

United States of America    851
United Kingdom              183
Canada                       74
Germany                      58
Netherlands                  47
Australia                    34
Sweden                       20
Ireland                      15
France                       14
Switzerland                  10
Brazil                       10
India                         9
Russia                        9
New Zealand                   9
Denmark                       7
Bulgaria                      7
Finland                       7
Belgium                       5
South Africa                  4
Poland                        4
Austria                       4
Czech Republic                3
Italy                         3
Chile                         3
Norway                        3
Romania                       3
Spain                         3
Other                         2
Bosnia and Herzegovina        2
Afghanistan                   2
Pakistan                      2
Israel  

In [23]:
# Determine which values to replace
replace_countries = list(country_counts[country_counts < 20].index)

# Replace in DataFrame
for country in replace_countries:
    mental_df['What country do you work in?'] = mental_df['What country do you work in?'].replace(country,"Other")

In [24]:
# function for comparing two columns with a chi-square-test

def chi_square(column):
    crosstab = pd.crosstab(mental_df['Have you been diagnosed with a mental health condition by a medical professional?'], mental_df[column])
    return stats.chi2_contingency(crosstab)[1]

In [25]:
# run chi-square test and drop non significant columns

insignificant_p_columns = []
for column in mental_df.columns:
    p_value = chi_square(column)
    if p_value >= .05:
        insignificant_p_columns.append(column)
        
insignificant_p_columns

['Are you self-employed?',
 'Did your previous employers provide resources to learn more about mental health issues and how to seek help?',
 'Do you think that discussing a physical health issue with previous employers would have negative consequences?',
 'Would you have been willing to discuss a mental health issue with your previous co-workers?',
 'Would you be willing to bring up a physical health issue with a potential employer in an interview?',
 'What is your age?',
 'Do you work remotely?']

In [26]:
# drop insignificant p value columns
mental_df.drop(columns=insignificant_p_columns, inplace=True)
mental_df.shape

(1433, 24)

In [27]:
mental_df


,Do you have previous employers?,Have your previous employers provided mental health benefits?,Were you aware of the options for mental health care provided by your previous employers?,Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?,Was your anonymity protected if you chose to take advantage of mental health or substance abuse treatment resources with previous employers?,Do you think that discussing a mental health disorder with previous employers would have negative consequences?,Would you have been willing to discuss a mental health issue with your direct supervisor(s)?,Did you feel that your previous employers took mental health as seriously as physical health?,Did you hear of or observe negative consequences for co-workers with mental health issues in your previous workplaces?,Would you bring up a mental health issue with a potential employer in an interview?,...,Do you have a family history of mental illness?,Have you had a mental health disorder in the past?,Do you currently have a mental health disorder?,Have you been diagnosed with a mental health condition by a medical professional?,Have you ever sought treatment for a mental health issue from a mental health professional?,"If you have a mental health issue, do you feel that it interferes with your work when being treated effectively?","If you have a mental health issue, do you feel that it interferes with your work when NOT being treated effectively?",What is your gender?,What country do you live in?,What country do you work in?
0,1,"No, none did",N/A (not currently aware),0,0,Some of them,Some of my previous employers,0,None of them,0,...,-1,1,-1,1,0,Not applicable to me,Not applicable to me,Male,United Kingdom,United Kingdom
1,1,"Yes, they all did",I was aware of some,None did,"Yes, always",None of them,Some of my previous employers,Some did,None of them,-1,...,1,1,1,1,1,Rarely,Sometimes,Male,United States of America,United States of America
2,1,"No, none did",N/A (not currently aware),None did,0,0,0,0,Some of them,1,...,-1,0,-1,-1,1,Not applicable to me,Not applicable to me,Other,United Kingdom,United Kingdom
3,1,Some did,N/A (not currently aware),None did,0,Some of them,Some of my previous employers,0,Some of them,0,...,-1,1,1,1,1,Sometimes,Sometimes,Male,United Kingdom,United Kingdom
4,1,0,N/A (not currently aware),Some did,0,Some of them,Some of my previous employers,Some did,Some of them,-1,...,1,1,1,1,1,Sometimes,Sometimes,Female,United States of America,United States of America
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1428,1,"Yes, they all did",I was aware of some,Some did,0,0,Some of my previous employers,Some did,None of them,-1,...,1,-1,-1,-1,1,Not applicable to me,Not applicable to me,Other,United States of America,United States of America
1429,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,...,1,-1,-1,1,0,Sometimes,Often,Other,United States of America,Other
1430,1,Some did,I was aware of some,None did,Sometimes,"Yes, all of them",Some of my previous employers,None did,Some of them,-1,...,1,1,0,1,1,Rarely,Sometimes,Male,United States of America,United States of America
1431,1,"No, none did",N/A (not currently aware),None did,0,"Yes, all of them","No, at none of my previous employers",None did,None of them,-1,...,1,0,1,1,0,Sometimes,Often,Female,United States of America,United States of America


In [28]:
mental_df.dropna(inplace=True)

In [29]:
mental_df.shape

(1199, 24)

In [30]:
mental_df.replace(",", "")

,Do you have previous employers?,Have your previous employers provided mental health benefits?,Were you aware of the options for mental health care provided by your previous employers?,Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?,Was your anonymity protected if you chose to take advantage of mental health or substance abuse treatment resources with previous employers?,Do you think that discussing a mental health disorder with previous employers would have negative consequences?,Would you have been willing to discuss a mental health issue with your direct supervisor(s)?,Did you feel that your previous employers took mental health as seriously as physical health?,Did you hear of or observe negative consequences for co-workers with mental health issues in your previous workplaces?,Would you bring up a mental health issue with a potential employer in an interview?,...,Do you have a family history of mental illness?,Have you had a mental health disorder in the past?,Do you currently have a mental health disorder?,Have you been diagnosed with a mental health condition by a medical professional?,Have you ever sought treatment for a mental health issue from a mental health professional?,"If you have a mental health issue, do you feel that it interferes with your work when being treated effectively?","If you have a mental health issue, do you feel that it interferes with your work when NOT being treated effectively?",What is your gender?,What country do you live in?,What country do you work in?
0,1,"No, none did",N/A (not currently aware),0,0,Some of them,Some of my previous employers,0,None of them,0,...,-1,1,-1,1,0,Not applicable to me,Not applicable to me,Male,United Kingdom,United Kingdom
1,1,"Yes, they all did",I was aware of some,None did,"Yes, always",None of them,Some of my previous employers,Some did,None of them,-1,...,1,1,1,1,1,Rarely,Sometimes,Male,United States of America,United States of America
2,1,"No, none did",N/A (not currently aware),None did,0,0,0,0,Some of them,1,...,-1,0,-1,-1,1,Not applicable to me,Not applicable to me,Other,United Kingdom,United Kingdom
3,1,Some did,N/A (not currently aware),None did,0,Some of them,Some of my previous employers,0,Some of them,0,...,-1,1,1,1,1,Sometimes,Sometimes,Male,United Kingdom,United Kingdom
4,1,0,N/A (not currently aware),Some did,0,Some of them,Some of my previous employers,Some did,Some of them,-1,...,1,1,1,1,1,Sometimes,Sometimes,Female,United States of America,United States of America
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1426,1,0,N/A (not currently aware),0,0,Some of them,Some of my previous employers,Some did,None of them,0,...,0,1,1,1,1,Rarely,Often,Female,Canada,Canada
1427,1,Some did,I was aware of some,None did,0,Some of them,"No, at none of my previous employers",None did,"Yes, all of them",-1,...,1,1,1,1,1,Rarely,Often,Female,United States of America,United States of America
1428,1,"Yes, they all did",I was aware of some,Some did,0,0,Some of my previous employers,Some did,None of them,-1,...,1,-1,-1,-1,1,Not applicable to me,Not applicable to me,Other,United States of America,United States of America
1430,1,Some did,I was aware of some,None did,Sometimes,"Yes, all of them",Some of my previous employers,None did,Some of them,-1,...,1,1,0,1,1,Rarely,Sometimes,Male,United States of America,United States of America


In [32]:
mental_df.to_csv('cleaned_data.csv', index=False)

In [33]:
print("create table cleaned_data (")
for column in mental_df.columns: 
    print("=", column.replace(" ", "_").upper(), "% varchar,")

create table cleaned_data (
= DO_YOU_HAVE_PREVIOUS_EMPLOYERS? % varchar,
= HAVE_YOUR_PREVIOUS_EMPLOYERS_PROVIDED_MENTAL_HEALTH_BENEFITS? % varchar,
= WERE_YOU_AWARE_OF_THE_OPTIONS_FOR_MENTAL_HEALTH_CARE_PROVIDED_BY_YOUR_PREVIOUS_EMPLOYERS? % varchar,
= DID_YOUR_PREVIOUS_EMPLOYERS_EVER_FORMALLY_DISCUSS_MENTAL_HEALTH_(AS_PART_OF_A_WELLNESS_CAMPAIGN_OR_OTHER_OFFICIAL_COMMUNICATION)? % varchar,
= WAS_YOUR_ANONYMITY_PROTECTED_IF_YOU_CHOSE_TO_TAKE_ADVANTAGE_OF_MENTAL_HEALTH_OR_SUBSTANCE_ABUSE_TREATMENT_RESOURCES_WITH_PREVIOUS_EMPLOYERS? % varchar,
= DO_YOU_THINK_THAT_DISCUSSING_A_MENTAL_HEALTH_DISORDER_WITH_PREVIOUS_EMPLOYERS_WOULD_HAVE_NEGATIVE_CONSEQUENCES? % varchar,
= WOULD_YOU_HAVE_BEEN_WILLING_TO_DISCUSS_A_MENTAL_HEALTH_ISSUE_WITH_YOUR_DIRECT_SUPERVISOR(S)? % varchar,
= DID_YOU_FEEL_THAT_YOUR_PREVIOUS_EMPLOYERS_TOOK_MENTAL_HEALTH_AS_SERIOUSLY_AS_PHYSICAL_HEALTH? % varchar,
= DID_YOU_HEAR_OF_OR_OBSERVE_NEGATIVE_CONSEQUENCES_FOR_CO-WORKERS_WITH_MENTAL_HEALTH_ISSUES_IN_YOUR_PREVIOUS_WORKP